# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,23.00,63,0,8.62,MU,1691608101
1,1,port-aux-francais,-49.3500,70.2167,2.48,78,87,12.06,TF,1691608021
2,2,beledweyne,4.7358,45.2036,26.00,60,79,9.93,SO,1691608619
3,3,nemuro,43.3236,145.5750,18.78,100,100,6.06,JP,1691607923
4,4,bubaque,11.2833,-15.8333,27.56,79,44,3.58,GW,1691608104


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
city_data_df.loc[city_data_df["Humidity"]<0]

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [22]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    frame_width=700,
    frame_height=500,
    global_extent = True,
    tiles = "OSM"
)

# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:

# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df=city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21)&(city_data_df["Wind Speed"]<4.5)&(city_data_df["Cloudiness"]==0), :]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna(how = 'any')


# Display sample data
narrowed_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
81,81,colonia,50.9333,6.9500,21.09,60,0,2.57,DE,1691607937
82,82,es castell,39.8795,4.2854,26.38,73,0,3.09,ES,1691608200
230,230,saint-jorioz,45.8333,6.1667,23.95,43,0,3.60,FR,1691608666
270,270,hauterive,46.0901,3.4459,25.42,63,0,2.22,FR,1691608674
273,273,longgang,37.6508,120.3286,24.39,86,0,4.08,CN,1691608675


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"].copy()]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

C:\Users\Eldiscala\AppData\Local\Temp\ipykernel_19284\1140153955.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
81,colonia,DE,50.9333,6.9500,60,
82,es castell,ES,39.8795,4.2854,73,
230,saint-jorioz,FR,45.8333,6.1667,43,
270,hauterive,FR,46.0901,3.4459,63,
273,longgang,CN,37.6508,120.3286,86,
310,morondava,MG,-20.2833,44.2833,86,
318,yssingeaux,FR,45.1333,4.1167,61,
341,elhovo,BG,42.1667,26.5667,67,
357,tarija,BO,-21.5355,-64.7296,24,
372,as-suwayda,SY,32.7090,36.5695,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accomodation.hotel", "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
colonia - nearest hotel: No hotel found
es castell - nearest hotel: No hotel found
saint-jorioz - nearest hotel: No hotel found
hauterive - nearest hotel: No hotel found
longgang - nearest hotel: No hotel found
morondava - nearest hotel: No hotel found
yssingeaux - nearest hotel: No hotel found
elhovo - nearest hotel: No hotel found
tarija - nearest hotel: No hotel found
as-suwayda - nearest hotel: No hotel found
aga - nearest hotel: No hotel found
uzgen - nearest hotel: No hotel found
susa - nearest hotel: No hotel found
hammerfest - nearest hotel: No hotel found
boueni - nearest hotel: No hotel found
zakynthos - nearest hotel: No hotel found
bunyan - nearest hotel: No hotel found
gori - nearest hotel: No hotel found
sona - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
81,colonia,DE,50.9333,6.9500,60,No hotel found
82,es castell,ES,39.8795,4.2854,73,No hotel found
230,saint-jorioz,FR,45.8333,6.1667,43,No hotel found
270,hauterive,FR,46.0901,3.4459,63,No hotel found
273,longgang,CN,37.6508,120.3286,86,No hotel found
310,morondava,MG,-20.2833,44.2833,86,No hotel found
318,yssingeaux,FR,45.1333,4.1167,61,No hotel found
341,elhovo,BG,42.1667,26.5667,67,No hotel found
357,tarija,BO,-21.5355,-64.7296,24,No hotel found
372,as-suwayda,SY,32.7090,36.5695,58,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    frame_width=700,
    frame_height=500,
    global_extent = True,
    tiles = "OSM", 
    hover_cols = ["Hotel Name", "Country"])
 
 
# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (Hotel Name,Country)